In [15]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC

from optuna_utils import spam_scorer

In [1]:
train_raw = pd.read_csv("data/sms_train.csv")
test_raw = pd.read_csv("data/sms_test.csv")
vectorizer = CountVectorizer(dtype=np.float64)
X_train = vectorizer.fit_transform(train_raw["message"])
y_train = train_raw["label"]
X_test = vectorizer.transform(test_raw["message"])

In [2]:
full_sms = pd.read_csv('data/sms.tsv', sep="\t", header=None)
full_sms.columns = ["label", "message"]
full_sms.label = full_sms.label.apply(lambda l: 1 if l == "spam" else 0)
full_sms

,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will ü b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [3]:
test_df = pd.merge(test_raw, full_sms.drop_duplicates(), on="message", how="inner")
test_df

,message,label
0,"Yo, you at jp and hungry like a mofo?",0
1,It's é only $140 ard...É rest all ard $180 at ...,0
2,"&lt;#&gt; , that's all? Guess that's easy enough",0
3,Y?WHERE U AT DOGBREATH? ITS JUST SOUNDING LIKE...,0
4,Good afternoon sexy buns! How goes the job sea...,0
...,...,...
995,Tell your friends what you plan to do on Valen...,0
996,No. Yes please. Been swimming?,0
997,Thank you. I like you as well...,0
998,Stupid.its not possible,0


In [4]:
y_test = test_df['label']

In [12]:
columns = np.array(pd.read_csv("data/rfe_svc_columns.csv", index_col=0, header=None)).reshape(-1,)
X_train_selected = X_train[:,columns]
X_test_selected = X_test[:,columns]

In [16]:
best_svc = LinearSVC(C=52.01425081408176, penalty='l1', dual=False, class_weight="balanced", max_iter=1_000_000).fit(X_train_selected, y_train)
spam_scorer(best_svc, X_test_selected, y_test)

0.5909827306883596

In [19]:
from xgboost import XGBClassifier
from sklearn.metrics import balanced_accuracy_score

clf = XGBClassifier().fit(X_train, y_train)
balanced_accuracy_score(clf.predict(X_test), y_test)

0.5991332269033381